In [14]:
import pandas as pd
from utils.utils import get_project_root
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
path_to_data = get_project_root() / "data"
df = pd.read_csv(path_to_data / "df.csv")
# turn date columns into datetime format
df[df.loc[:, df.columns.str.contains("date")].columns] = df.loc[:, df.columns.str.contains("date")].apply(
    pd.to_datetime)
# turn language and source_domain into categoricals
df[["language", "source_domain"]] = df[["language", "source_domain"]].astype("category")
# convert text-columns into text (from object
df[["description", "filename", "title", "title_page", "title_rss", "text_main"]].astype("str")

,description,filename,title,title_page,title_rss,text_main
0,Die Essayistin und Schriftstellerin Joan Didio...,aktuell_feuilleton_buecher_autoren_schriftstel...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Die Essayistin und Schriftstellerin Joan Didio...
1,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,aktuell_feuilleton_buecher_rezensionen_sachbuc...,J.R. Moores' Buch Electric Wizards,J.R. Moores' Buch Electric Wizards,J.R. Moores' Buch Electric Wizards,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...
2,"Auf einer neuen Dating-Seite suchen Einsame, d...",aktuell_feuilleton_dating-seite-fuer-impfgegne...,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Unterwegs auf Impffrei:Love: Die ungeimpfte Liebe,"Die Menschen wollen lieben. Selbst die, die ha..."
3,Drehungen eines Unbelehrbaren: Nach Kritik an ...,aktuell_feuilleton_debatten_frank-ulrich-montg...,Frank Ulrich Montgomery verteidigt seine Richt...,Frank Ulrich Montgomery verteidigt seine Richt...,Montgomerys Mätzchen: Ihr seid ja nur eifersüc...,Ungern möchte man stören bei dem langen Lauf z...
4,Der Ethikrat befürwortet mehrheitlich eine all...,aktuell_feuilleton_debatten_impfpflicht-jetzt-...,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht oder nicht: Wir müssen miteinander...,Als am 1. Januar 2019 Heribert Prantl in seine...
...,...,...,...,...,...,...
989,Das James Webb-Weltraumteleskop ist nach lan...,wissenschaft_video235866470_Hubble-Nachfolger-...,Hubble-Nachfolger: Diese Fragen soll das Webb-...,Hubble-Nachfolger: Diese Fragen soll das Webb-...,James Webb-Teleskop  Hubble-Nachfolger ins ...,Das James Webb-Weltraumteleskop ist nach lan...
990,Das James Webb-Teleskop ist erfolgreich ins ...,wissenschaft_video235869636_Weltraum-Teleskop-...,Weltraum-Teleskop James Webb: Astronaut Walt...,Weltraum-Teleskop James Webb: Astronaut Walt...,Da kann noch viel schiefgehen,Das James Webb-Teleskop ist erfolgreich ins ...
991,Eine Studie des Universitätsklinikums Hamburg-...,wissenschaft_video236043420_Studie-aus-Hamburg...,Studie aus Hamburg: Organschäden auch bei mild...,Studie aus Hamburg: Organschäden auch bei mild...,Organschäden auch bei mildem Corona-Verlauf,Eine Studie des Universitätsklinikums Hamburg-...
992,Nach gut 30 Jahren Planung war es so weit: Ein...,wissenschaft_weltall_james-webb-weltraumtelesk...,James-Webb-Weltraumteleskop ist ins All gestartet,James-Webb-Weltraumteleskop ist ins All gestar...,James-Webb-Weltraumteleskop ist ins All gestartet,»Hubble«-Nachfolger Weltraumteleskop »James We...


In [12]:
import math
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
# WARNING - 2.24 GB model size!
tagger = SequenceTagger.load("flair/ner-german-large")
articles = {"texts": df[["title", "title_page", "title_rss", "description", "text_main"]]}
entities = []
for article in [x[1] for x in articles["texts"].iterrows()]:
    entity = {}
    for k in ["title", "description", "text_main"]:
        try:
            entity[k] = str(article[k]).replace("\x84", '"').replace("\x93", '"')
            chunks = [article[k]]
            if len(article[k]) > 512:
                # brute-force chunk the article
                chunks = [article[k][512 * (x - 1):512 * x] for x in range(1, math.ceil(len(article[k]) / 512))]
            chunks_ner = []
            for chunk in chunks:
                sentence = Sentence(chunk)
                tagger.predict(sentence)
                chunks_ner.append(sentence.to_dict(tag_type='ner'))
            entity[k + "_entities"] = chunks_ner
        except:
            pass
    entities.append(entity)
ent = pd.DataFrame(entities)
# ent.to_csv("entities.csv")
ent

2022-01-08 12:03:57,660 loading file C:\Users\LeonWolf\.flair\models\ner-german-large\6b8de9edd73722050be2547acf64c037b2df833c6e8f0e88934de08385e26c1e.4b0797effcc6ebb1889d5d29784b97f0a099c1569b319d87d7c387e44e2bba48


,title,title_entities,description,description_entities,text_main,text_main_entities
0,Schriftstellerin Joan Didion mit 87 Jahren ges...,[{'text': 'Schriftstellerin Joan Didion mit 87...,Die Essayistin und Schriftstellerin Joan Didio...,[{'text': 'Die Essayistin und Schriftstellerin...,Die Essayistin und Schriftstellerin Joan Didio...,[{'text': 'Die Essayistin und Schriftstellerin...
1,"J.R. Moores' Buch ""Electric Wizards""",[{'text': 'J.R. Moores' Buch „Electric Wizards...,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,[{'text': 'Heute schon Pigs Pigs Pigs Pigs Pig...,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,[{'text': 'Heute schon Pigs Pigs Pigs Pigs Pig...
2,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,[{'text': 'Dating-Seite für Impfgegner: Die un...,"Auf einer neuen Dating-Seite suchen Einsame, d...",[{'text': 'Auf einer neuen Dating-Seite suchen...,"Die Menschen wollen lieben. Selbst die, die ha...",[{'text': 'Die Menschen wollen lieben. Selbst ...
3,Frank Ulrich Montgomery verteidigt seine Richt...,[{'text': 'Frank Ulrich Montgomery verteidigt ...,Drehungen eines Unbelehrbaren: Nach Kritik an ...,[{'text': 'Drehungen eines Unbelehrbaren: Nach...,Ungern möchte man stören bei dem langen Lauf z...,[{'text': 'Ungern möchte man stören bei dem la...
4,Impfpflicht jetzt? Wir müssen miteinander reden!,[{'text': 'Impfpflicht jetzt? Wir müssen mitei...,Der Ethikrat befürwortet mehrheitlich eine all...,[{'text': 'Der Ethikrat befürwortet mehrheitli...,Als am 1. Januar 2019 Heribert Prantl in seine...,[{'text': 'Als am 1. Januar 2019 Heribert Pran...
...,...,...,...,...,...,...
989,Hubble-Nachfolger: Diese Fragen soll das Webb-...,[{'text': 'Hubble-Nachfolger: Diese Fragen sol...,"Das ""James Webb""-Weltraumteleskop ist nach lan...",[{'text': 'Das „James Webb“-Weltraumteleskop i...,"Das ""James Webb""-Weltraumteleskop ist nach lan...",[{'text': 'Das „James Webb“-Weltraumteleskop i...
990,"Weltraum-Teleskop ""James Webb"": Astronaut Walt...",[{'text': 'Weltraum-Teleskop „James Webb“: Ast...,"Das ""James Webb""-Teleskop ist erfolgreich ins ...",[{'text': 'Das „James Webb“-Teleskop ist erfol...,"Das ""James Webb""-Teleskop ist erfolgreich ins ...",[{'text': 'Das „James Webb“-Teleskop ist erfol...
991,Studie aus Hamburg: Organschäden auch bei mild...,[{'text': 'Studie aus Hamburg: Organschäden au...,Eine Studie des Universitätsklinikums Hamburg-...,[{'text': 'Eine Studie des Universitätskliniku...,Eine Studie des Universitätsklinikums Hamburg-...,[{'text': 'Eine Studie des Universitätskliniku...
992,James-Webb-Weltraumteleskop ist ins All gestartet,[{'text': 'James-Webb-Weltraumteleskop ist ins...,Nach gut 30 Jahren Planung war es so weit: Ein...,[{'text': 'Nach gut 30 Jahren Planung war es s...,»Hubble«-Nachfolger Weltraumteleskop »James We...,[{'text': '»Hubble«-Nachfolger Weltraumtelesko...


In [13]:
ent.to_csv("entities.csv")

In [11]:
ent["text_main_entities"][0]

[{'text': 'Die Essayistin und Schriftstellerin Joan Didion ist gestorben. Sie gehörte zu den wichtigsten Vertretern des amerikanischen „New Journalism“. Die Essayistin und Schriftstellerin Joan Didion ist tot. Sie starb am Donnerstag im Alter von 87 Jahren in ihrem Haus in Manhattan, wie ihr Verleger mitteilte. Didion schenkte dem amerikanischen Roman einen unverwechselbaren Ton. Neben Tom Wolfe, Truman Capote und Norman Mailer gehörte sie zu den berühmtesten Vertretern des „New Journalism“, einer radikal subjektiven,',
  'labels': [],
  'entities': [{'text': 'Joan Didion',
    'start_pos': 36,
    'end_pos': 47,
    'labels': [PER (1.0)]},
   {'text': 'Joan Didion',
    'start_pos': 178,
    'end_pos': 189,
    'labels': [PER (1.0)]},
   {'text': 'Manhattan',
    'start_pos': 263,
    'end_pos': 272,
    'labels': [LOC (1.0)]},
   {'text': 'Didion', 'start_pos': 302, 'end_pos': 308, 'labels': [PER (1.0)]},
   {'text': 'Tom Wolfe',
    'start_pos': 378,
    'end_pos': 387,
    'labels'